pourquoi?

In [ ]:
# basic imports
import os
import matplotlib.pyplot as plt
# display matplotlib graphics in notebook
%matplotlib inline 
import seaborn as sns

# disable warnings for libraries
import warnings
warnings.filterwarnings("ignore")

# configure logger
import logging
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%I:%M:%S')
logger = logging.getLogger(__name__)


In [ ]:
# Installation : pip install shap
import shap

# Cette fonction permet de lier le texte brut à la décision du modèle
def predict_proba(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        # On utilise le vecteur CLS pour la prédiction finale (à adapter selon votre classifieur)
        emb = outputs.last_hidden_state[:, 0, :].cpu().numpy()
    return clf.predict_proba(emb)

# Expliquer une prédiction sur un manifeste spécifique
explainer = shap.Explainer(predict_proba, tokenizer)
shap_values = explainer(df_final['clean_text'].iloc[:5]) # Analyse des 5 premiers

# Visualisation (génère un graphique interactif en couleurs)
shap.plots.text(shap_values[0])

Le Support (titulaire-support) : Maintenant que vous avez features (vos X) et df_final['titulaire-support'] (vos Y), vous pouvez entraîner un classifieur classique très puissant comme un Random Forest ou un SVM.

Gestion de la longueur : Le paramètre truncation=True est vital. Les manifestes de 81 sont longs ; ici, on ne garde que les 512 premiers tokens (environ les 300 premiers mots) pour l'embedding. Souvent, cela suffit largement à identifier la couleur politique d'un texte.

Vitesse : L'utilisation de batch_size=16 permet d'accélérer le calcul par rapport à une boucle ligne par ligne.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Définition des cibles (étiquettes politiques)
y = df_final['titulaire-support']

# Split Entraînement / Test
X_train, X_test, y_train, y_test = train_test_split(features.numpy(), y, test_size=0.2)

# Entraînement
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

print(f"Score de précision sur 1981 : {clf.score(X_test, y_test):.2%}")